In [1]:
from experiment import *
import os

/Users/sebastianrueclausen/Code/fugal/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
experiment = Experiment(
    graphs=[
        Graph(GraphKind.LOBSTER, {
            'n': 64,
            'p1': 0.2,
            'p2': 0.2,
        }),
    ],
    algorithms=[
        Algorithm(Config(), use_fugal=False),
    ],
    noise_levels=[
        NoiseLevel(0.05, 0.0, False),
        NoiseLevel(0.1, 0.0, False),
    ],
)
results = experiment.run()

folder = "results"
if not os.path.exists(folder): os.makedirs(folder)
results.dump(folder)


λ: 100%|██████████| 15/15 [00:00<00:00, 30.71it/s]
